In [27]:
import pandas as pd
import plotly.graph_objects as go
#%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio

In [28]:
def generate_links_uncoded(file_path = "baseline_flow.csv", redirect = True):
    df = pd.read_csv(file_path)
    
    if redirect == True:
        to_recycle = 0
        to_remelt = 0
        for idx in df[(df['source'].str.contains('F|P', na=False))&(df['target'] == 'R2')].index: 
            to_recycle += df.loc[idx,'amount']
            df.loc[idx,'target'] = 'P9'
        df.loc[len(df)] = ['P9','R2',to_recycle,'Recycle']

        for idx in df[(df['source'].str.contains('P5|P6|P7|P8', na=False))&(df['target'] == 'R1')].index: 
            to_remelt += df.loc[idx,'amount']
            df.loc[idx,'target'] = 'P10'
        df.loc[len(df)] = ['P10','R1',to_remelt,'Recycle']
            
        
    ### Flow Color
    color_map = {"Alumina": "darkorange",
                 "Bauxite": "darkred",
                "Aluminum_primary" : "blueviolet",
                 "Aluminum_secondary" : "plum",
                "Slab":"deeppink",
                "Billet":"deeppink",
                "Wrought_ingot":"deeppink",
                "Cast_ingot":"pink",
                "Sheets":"seagreen",
                 "Extrusions":"seagreen",
                 "Cables":"seagreen",
                 "Forgings":"seagreen",
                "Castings":"lightgreen",
                 'Ingot': 'hotpink',
                 'Forming':'green',
                "Scrap":"lightgrey",
                 "Recycle":"lightgrey",
                "Waste":"darkred",
                "Products":"dodgerblue"}
    
    for row in df.index:
        if df.loc[row,"type"] in color_map:
            df.loc[row,"color"] = color_map[df.loc[row,"type"]]
        else:
            df.loc[row,"color"] = color_map["Products"]

    
    return df

In [29]:
df = generate_links_uncoded()

In [30]:
def generate_node_table():
    node_table = pd.read_excel("inputs.xlsx", sheet_name = 'Process',usecols = 'A:H',nrows = 24)
    node_table.set_index("Label", inplace = True)
    node_table = node_table.drop('W')
    node_table.drop(columns = ['CF_Import','CF Ref'],inplace = True)

    ### The following calculation will be based on links_uncoded, NOT links
    for key in node_table.index:
        inflow = 0
        outflow = 0
        valuable = 0
        loop = 0
        for row in links_uncoded.index:
            if links_uncoded.loc[row,"target"] == key:
                inflow = inflow + links_uncoded.loc[row,"amount"]
                if links_uncoded.loc[row,"type"] == 'Recycle':
                    loop = loop + links_uncoded.loc[row,"amount"]
            if links_uncoded.loc[row,"source"] == key:
                outflow = outflow + links_uncoded.loc[row,"amount"]
            if links_uncoded.loc[row,"source"] == key and links_uncoded.loc[row,"type"] not in ['Waste','Recycle']:
                valuable = valuable + links_uncoded.loc[row,"amount"]

                
            
                

        node_table.loc[key,"inflow"] = inflow
        node_table.loc[key,"outflow"] = outflow
        node_table.loc[key,"valuable"] = valuable
        node_table.loc[key,"loop"] = loop
            
    # Export
    inflow = 0
    for target in ['Export_Oxide','Export_Scrap','Export_Prod', 'Export_Form', 'Export_Cast', 'Export_Fabr']:
        inflow = links_uncoded[links_uncoded['target'] == target]['amount'].sum()
        if inflow > 1:
            node_table.loc[target,'inflow'] = inflow
        
    #import
    inflow = 0
    for source in ['Import_Ore','Import_Oxide','Import_EoL','Import_Metal',
                   'Import_Metal_C','Import_Form','Import_Fabr','Import_Prod','EoL_0','P9','P10']:
        inflow = links_uncoded[links_uncoded['source'] == source]['amount'].sum()
        
        if inflow > 1:
            node_table.loc[source,'inflow'] = inflow
            
    #waste
    inflow = 0
    for target in ['W_M','W_E','W_R1','W_R2','W_SC1','W_SC2','W_SC3','P9','P10']:
        inflow = links_uncoded[links_uncoded['target'] == target]['amount'].sum()
        if inflow > 1:
            node_table.loc[target,'inflow'] = inflow
            
    
    
    
    # Coordination
    ### R2 should not be too high or too low!
    import_y = 1
    export_y = 1.06
    x_0 = 0.01
    x_1 = 0.1
    x_2 = 0.2
    x_3 = 0.425
    x_4 = 0.55
    x_5 = 0.75
    x_6 = 1
    x_r = 0.3
    position_mapping = {"M":[x_1,0.01],
             'EoL_0': [x_0, 0.4],"EoL":[x_1, 0.405],
             "E":[x_2, 0.01],
             "R1":[x_r, 0.3], "R2":[x_r, 0.7],
             "C1":[x_3, 0.01], "C2":[x_3, 0.7],
             "F1":[x_4, 0.01], "F2":[x_4, 0.2], "F3":[x_4, 0.3], "F4":[x_4, 0.4],
             "SC1":[x_4, 0.6], "SC2":[x_4, 0.7], "SC3":[x_4, 0.8],
             'P1': [x_5, 0.10],
             'P9': [x_5 +0.1, 0.78],
             'P10': [x_5 +0.1, 0.01],
             "Use":[x_6, 0.15],
             "W":[0.35,0.5],
             'Import_Ore':[x_0,import_y],'Import_EoL': [x_0,import_y+0.06],
             'Import_Oxide':[x_1,import_y], 'Import_Metal':[x_2,import_y], 'Import_Metal_C':[x_r,import_y],
             'Import_Cast': [x_2,import_y], 'Import_Form': [x_3, import_y],
             'Import_Fabr': [x_4, import_y],'Import_Prod':[x_5,import_y],
             'Export_Cast': [x_4,export_y], 'Export_Form': [x_5,export_y],
             'Export_Proc':[x_3,export_y],'Export_Prod':[x_6,export_y],
             'Export_Oxide':[x_2, export_y],'Export_Scrap':[x_r, export_y]                        
             }

    for i in range(2,9):
        position_mapping['P'+str(i)] = [x_5,i*0.1-0.01]
        
    for idx in ['M','E','R1','R2','SC1','SC2','SC3']:
        position_mapping['W_'+idx] = [position_mapping[idx][0]+0.02, position_mapping[idx][1]+0.06]


    for row in node_table.index:
        node_table.loc[row,"x"] = position_mapping[row][0]
        node_table.loc[row,"y"] = position_mapping[row][1]
        
    # Label
    label_mapping = {'Import_Ore':'Import:Bauxite',
        'Import_Oxide': 'Import:Alumina','Import_EoL': 'Import:Scrap', 
        'Import_Metal':'Import:Metal','Import_Metal_C':'Import:Metal',
        'Import_Form':'Import:Alloy',
        'Import_Fabr':'Import:Semi','Import_Prod':'Import:Product',
        'Export_Oxide':'Export:Alumina','Export_Scrap':'Export:Scrap','Export_Cast':'Export:Alloy',
        'Export_Form':'Export:Semi','Export_Prod':'Export:Product'          
    }
    for idx in label_mapping:
        node_table.loc[idx,'Display'] = label_mapping[idx]
    
    node_table.fillna(0, inplace = True)
    return node_table

In [31]:
def set_display(node_table, all_label = True, show_node_label = True):
    df = node_table
    display = []
    for key in node_table.index:
        decimal = None #1

        flow_label = df.loc[key,'Display']
        # Add node label to the left of output flow label
        if show_node_label == True:            
            if len(key) <= 3 and key not in ['Use','EoL','M']:
                flow_label = node_label = f'{key}-' + str(flow_label)
            if key == 'M':
                flow_label = f'B-{flow_label}'
            if key == 'EoL':
                flow_label = 'S-' + str(flow_label)
            if key == 'Use':
                flow_label = f'{flow_label}<br>(Use)'
        
        if key in ['EoL']:
            display.append(flow_label + "<br>"+str(round(links_uncoded[links_uncoded["source"]== key]["amount"].sum(),decimal)))
            
        elif key in ["C1","C2"]:
            display.append(flow_label + "<br>"+str(round(node_table.loc[key,"outflow"],decimal)))
            
        
        elif key in ["M","E","R1","R2",
                     "F1","F2","F3", "F4",
                   "P1","P2","P3","P4","P5","P6","P7","P8"]:
            display.append(flow_label + "<br>"+str(round(node_table.loc[key,"valuable"],decimal)))
            
        elif key in ["SC1","SC2","SC3"]:
            display.append(flow_label + "<br>"+str(round(node_table.loc[key,"valuable"],decimal)))
            
        elif key in ['Use']:
            display.append(flow_label + "<br>"+str(round(node_table.loc[key,"inflow"],decimal)))
            
        elif 'Export' in key:
            if all_label == True:
                display.append(flow_label + "<br>"+str(round(node_table.loc[key,"inflow"],decimal)))
            else:
                display.append(str(round(node_table.loc[key,"inflow"],decimal)))
            
            
        elif 'Import' in key:
            if all_label == True:
                display.append(flow_label + "<br>"+str(round(node_table.loc[key,"inflow"],decimal)))
            else:
                display.append("+" + str(round(node_table.loc[key,"inflow"],decimal)))
                
                
        
        elif 'W' in key:
            #display.append(str(round(node_table.loc[key,"inflow"],decimal)))
            display.append(f'Loss: {round(node_table.loc[key,"inflow"],decimal)}')
        
        elif key == 'EoL_0':
            display.append('US Collected<br>EoL Scrap<br>'+str(round(node_table.loc[key,"inflow"],decimal)))

        elif key in ['P9','P10']:
            display.append(None)
                                      
    return display

In [32]:
def generate_links():
    links = links_uncoded.copy()
    links["line_width"] = 0.00000

    # Operation below will not affect labeled number
    for row in links.index:
        if links.loc[row,"amount"] < 5:
            if 'Import' in links.loc[row, 'source'] or 'Export' in links.loc[row, 'target']:
                links.loc[row,"amount"] = 0.0000001
                
                
    
    label = list(node_table.index)
    mapping_dict = {x:y for y,x in enumerate(label)}
    
    links["source"] = links["source"].map(mapping_dict)
    links["target"] = links["target"].map(mapping_dict)
    
    

            
    return links

In [33]:
def generate_figure_object(show_labels = True):
    links_dict = links.to_dict(orient = 'list')
    fig = go.Figure(data = go.Sankey(
        
        node = dict(pad = 1, thickness = 5, line = dict(color = "white", width = 0.01),
                   label=display if show_labels else None,
                   color = font_color_list,
                   x = node_table["x"],
                   y = node_table["y"]),
        arrangement = 'snap', #"freeform","fixed",'perpendicular'

        link = dict(source = links_dict["source"],
                   target = links_dict["target"],
                   value = links_dict["amount"],
                   color = links_dict["color"],
                   #line = dict(color = "white", width = links["line_width"])
                   )
    ))
    return fig

In [34]:
links_uncoded = generate_links_uncoded(file_path = "test_flow.csv")
links_uncoded
# Export file
links_uncoded.to_csv("flows.csv", index = False)

In [35]:
links_uncoded

,source,target,amount,type,color
0,M,E,385.411765,Alumina,darkorange
1,M,W_M,37.814706,Waste,darkred
2,EoL,R1,2220.930961,Scrap,lightgrey
3,EoL,R2,1638.002424,Scrap,lightgrey
4,E,R1,31.353004,Aluminum_primary,blueviolet
...,...,...,...,...,...
156,SC1,SC1,196.930026,Recycle,lightgrey
157,SC2,SC2,135.060087,Recycle,lightgrey
158,SC3,SC3,61.784120,Recycle,lightgrey
159,P9,R2,352.753440,Recycle,lightgrey


In [36]:
links_uncoded[links_uncoded['source'] == 'Import_EoL']

,source,target,amount,type,color
106,Import_EoL,EoL,1369.242072,Scrap,lightgrey


In [37]:
node_table = generate_node_table()   
node_table

,Process,Stage,Output,Input,Display,inflow,outflow,valuable,loop,x,y
Label,,,,,,,,,,,
M,Bayer Process,Source,Alumina,Bauxite,Alumina,726.050000,726.050000,688.235294,0.000000,0.100,0.010
EoL,EoL Scrap Processing,Source,Scrap,Scrap,External Scrap,5753.614963,5753.614963,5753.614963,0.000000,0.100,0.405
E,Electrolysis,Processing,Aluminum_primary,Alumina,Primary Metal,1025.031176,1025.031176,1010.000000,0.000000,0.200,0.010
R1,Remelting,Processing,Aluminum_secondary,Primary Metal,Remelted Metal,6812.741989,6812.741989,6472.104890,3174.780956,0.300,0.300
R2,Refining,Processing,Aluminum_secondary,Primary Metal,Recycled Metal,2100.507374,2100.507374,1995.482006,352.753440,0.300,0.700
C1,Wrought Ingot Casting,Casting,Wrought_ingot,Primary Metal,Wrought Alloys,7353.640048,7353.640048,7353.640048,0.000000,0.425,0.010
C2,Foundry Ingot Casting,Casting,Cast_ingot,Primary Metal,Foundry Alloys,2280.565064,2280.565064,2280.565064,0.000000,0.425,0.700
F1,Rolling,Forming,Sheets,Wrought Alloys,Sheets and Foils,5688.063061,5688.063061,4025.328467,0.000000,0.550,0.010
F2,Extrusion,Forming,Extrusions,Wrought Alloys,Extrusions,2396.199984,2396.199984,1769.713498,0.000000,0.550,0.200


In [38]:
display = set_display(node_table)
display

['B-Alumina<br>688',
 'S-External Scrap<br>5754',
 'E-Primary Metal<br>1010',
 'R1-Remelted Metal<br>6472',
 'R2-Recycled  Metal<br>1995',
 'C1-Wrought Alloys<br>7354',
 'C2-Foundry Alloys<br>2281',
 'F1-Sheets and Foils<br>4025',
 'F2-Extrusions<br>1770',
 'F3-Cables/Wires<br>349',
 'F4-Forgings and Powders<br>132',
 'SC1-Die Castings<br>1092',
 'SC2-Permanent Mold Castings<br>749',
 'SC3-Sand Castings<br>342',
 'P1-Construction<br>1173',
 'P2-Transportation (Auto)<br>1778',
 'P3-Transportation (Other)<br>875',
 'P4-Durables<br>575',
 'P5-Electrical<br>726',
 'P6-Machinery<br>612',
 'P7-Containers<br>1700',
 'P8-Other Products<br>271',
 'Domestic Consumption<br>(Use)<br>9021',
 'Export:Alumina<br>303',
 'Export:Scrap<br>1895',
 'Export:Product<br>1235',
 'Export:Semi<br>853',
 'Export:Alloy<br>205',
 'Import:Bauxite<br>726',
 'Import:Alumina<br>640',
 'Import:Scrap<br>1369',
 'Import:Metal<br>1464',
 'Import:Metal<br>219',
 'Import:Alloy<br>1574',
 'Import:Semi<br>1186',
 'Import:Prod

In [39]:
def node_color(color_coded):
    
    font_color_list = []
    if color_coded == True:
        for idx in node_table.index:
        
            if idx in ['Import_Oxide','Import_EoL','Export_Oxide','Export_Scrap',
                      'Import_Fabr','Export_Cast','Export_Form','E','M']:
                font_color_list.append('red')
            elif node_table.loc[idx,'Stage'] in ['Forming','Fabrication']:
                font_color_list.append('blue')
            elif idx in ['Import_Prod','Export_Prod','Use']:
                font_color_list.append('blue')
            else:
                font_color_list.append('black')
    else:
        for idf in node_table.index:
            font_color_list.append('black')

    for p in ["P9", "P10"]:
        font_color_list[node_table.index.get_loc(p)] = "lightgray"
    
    return font_color_list

In [40]:
links = generate_links()
links

,source,target,amount,type,color,line_width
0,0,2,385.411765,Alumina,darkorange,0.0
1,0,39,37.814706,Waste,darkred,0.0
2,1,3,2220.930961,Scrap,lightgrey,0.0
3,1,4,1638.002424,Scrap,lightgrey,0.0
4,2,3,31.353004,Aluminum_primary,blueviolet,0.0
...,...,...,...,...,...,...
156,11,11,196.930026,Recycle,lightgrey,0.0
157,12,12,135.060087,Recycle,lightgrey,0.0
158,13,13,61.784120,Recycle,lightgrey,0.0
159,37,4,352.753440,Recycle,lightgrey,0.0


In [41]:
font_color_list = node_color(True)

### Single_run.py

In [42]:
links_uncoded = generate_links_uncoded("test_flow.csv", redirect = True)
node_table = generate_node_table()
display = set_display(node_table)
links = generate_links()
### No color-coding (False) for paper
font_color_list = node_color(False)
### With color-coding (True) for SI
#font_color_list = node_color(True)

In [43]:
fig = generate_figure_object()

R1_pre = int(node_table.loc['R1','loop']) #R1.inflow_table.loc[R1.inflow_table['type'] == 'Recycle', 'amount'].sum()
R2_pre = int(node_table.loc['R2','loop']) #R2.inflow_table.loc[R2.inflow_table['type'] == 'Recycle', 'amount'].sum()

fig.update_layout(font = dict(size=11, color= 'black'),hovermode='x',margin=dict(l=0, r=5, t=60, b=50), width = 1980*0.5, height = 1080*0.5)
fig.add_annotation(dict(font=dict(color="black",size=12), x=0.77, y=1.11, showarrow=False, text = 'Fabrication Scrap'))
fig.add_annotation(dict(font=dict(color="black",size=12), x=0.5, y=1.09, showarrow=False, text = 'Forming Scrap'))
fig.add_annotation(dict(font=dict(color="black",size=12), x=0.825, y=0.11, showarrow=False, text = f'Forming & Fabrication Scrap'))
fig.add_annotation(dict(font=dict(color="black",size=12), x=0.425, y=1.10, showarrow=False, text = f'{R1_pre:d}'))
fig.add_annotation(dict(font=dict(color="black",size=12), x=0.425, y=0.11, showarrow=False, text = f'{R2_pre:d}'))


fig.show()

pio.write_image(fig, 'single_run.png',scale = 8, width = 1980*0.6, height = 1080*0.6)

## Concept Art

In [44]:
fig = generate_figure_object(show_labels = False)


fig.update_layout(font = dict(size=11, color= 'black'),hovermode='x',margin=dict(l=0, r=5, t=60, b=50), width = 1980*0.5, height = 1080*0.5)

fig.show()

pio.write_image(fig, 'conceptual_sankey.png',scale = 8, width = 1980*0.6, height = 1080*0.6)

### Baseline 2050

In [45]:
links_uncoded = generate_links_uncoded("baseline_flow.csv", redirect = True)
node_table = generate_node_table()
display = set_display(node_table)
links = generate_links()
font_color_list = node_color(False)

In [ ]:
fig = generate_figure_object()

fig.update_layout(font = dict(size=12, color='black'),hovermode='x',margin=dict(l=0, r=5, t=60, b=50), width = 1980*0.5, height = 1080*0.5)

fig.show()

pio.write_image(fig, "baseline.png",scale = 8, width = 1980*0.6, height = 1080*0.6)

## Options

### Option 1: Improving yield efficiencies

In [ ]:
option_idx = 1
links_uncoded = generate_links_uncoded("option"+str(option_idx)+"_flow.csv", redirect = True)
node_table = generate_node_table()
display = set_display(node_table)
links = generate_links()
font_color_list = node_color(False)

In [ ]:
fig = generate_figure_object()

fig.update_layout(font = dict(size=12, color='black'),hovermode='x',margin=dict(l=0, r=5, t=60, b=50), width = 1980*0.5, height = 1080*0.5)

fig.show()

pio.write_image(fig, "option"+str(option_idx)+".png",scale = 8, width = 1980*0.6, height = 1080*0.6)

### Option 2: Optimize EoL recycling

In [ ]:
option_idx = 2
links_uncoded = generate_links_uncoded("option"+str(option_idx)+"_flow.csv", redirect = True)
node_table = generate_node_table()

display = set_display(node_table)
links = generate_links()
font_color_list = node_color(False)

In [ ]:
fig = generate_figure_object()

fig.update_layout(font = dict(size=12, color='black'),hovermode='x',margin=dict(l=0, r=5, t=60, b=50), width = 1980*0.5, height = 1080*0.5)

fig.show()

pio.write_image(fig, "option"+str(option_idx)+".png",scale = 8, width = 1980*0.6, height = 1080*0.6)

### Special Scenarios: Optimal, Lightweight, Island
For optimal simulate it using `single_run.py`, with `reset_system()` executed.

In [ ]:
links_uncoded = generate_links_uncoded("test_flow.csv", redirect = True)
node_table = generate_node_table()
display = set_display(node_table)
links = generate_links()
font_color_list = node_color(False)

In [ ]:
fig = generate_figure_object()

fig.update_layout(font = dict(size=12, color='black'),hovermode='x',margin=dict(l=0, r=5, t=60, b=50), width = 1980*0.5, height = 1080*0.5)

fig.show()

pio.write_image(fig, 'special.png',scale = 8, width = 1980*0.6, height = 1080*0.6)